In [ ]:
# %pip install FastAPI
# %pip install fastapi-offline
# %pip install uvicorn

In [1]:
from fastapi_offline import FastAPIOffline as FastAPI
from fastapi import HTTPException
from pydantic import BaseModel
import sqlite3

In [2]:
app = FastAPI()

In [3]:
@app.get('/')
async def read_root():
    return {'message': 'Hello, FastAPI with offline capabilities!'}

class Book(BaseModel):
    id: int
    title: str
    author: str
    year: int
    
def getdb():
    conn = sqlite3.connect('books.db')
    conn.row_factory = sqlite3.Row
    return conn

def initdb():
    conn = getdb()
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS books (
        id INTEGER PRIMARY KEY, 
        title TEXT NOT NULL, 
        author TEXT NOTEND, 
        year INTEGER NOT NULL
        )
        ''')
    conn.commit()
    conn.close()

# initdb()
 
@app.post('/book')
def addnewbook(book: Book):
    try:
        conn = getdb()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO books (title, author, year) VALUES (?,?,?)", (book.title, book.author, book.year))
        conn.commit()
        conn.close()
        return {'message': 'Book added successfully', 'body': book}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error {str(e)}")


@app.get('/books')
def getallbooks():
    conn = getdb()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books")
    rows = cursor.fetchall()
    books = []
    for row in rows:
        books.append(Book(id=row['id'], title=row['title'], author=row['author'], year=row['year']))
    conn.close()
    return books

@app.get('/book/{book_id}')
def getbookbyid(book_id: int):
    conn = getdb()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books WHERE id=?", (book_id,))
    row = cursor.fetchone()
    if row is None:
        raise HTTPException(status_code=404, detail="Book not found")
    conn.close()
    return Book(id=row['id'], title=row['title'], author=row['author'], year=row['year'])

@app.put('/book/{book_id}')
def updatebook(book_id: int, book: Book):
    try:
        conn = getdb()
        cursor = conn.cursor()
        cursor.execute("UPDATE books SET title=?, author=?, year=? WHERE id=?", (book.title, book.author, book.year, book_id))
        conn.commit()
        conn.close()
        return {'message': 'Book updated successfully', 'body': book}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error {str(e)}")

@app.delete('/book/{book_id}')
def deletebook(book_id: int):
    try:
        conn = getdb()
        cursor = conn.cursor()
        cursor.execute("DELETE FROM books WHERE id=?", (book_id,))
        conn.commit()
        conn.close()
        return {'message': 'Book deleted successfully'}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error {str(e)}")


In [4]:
import asyncio
import uvicorn

# Configure the server with specific host (IP) and port
config = uvicorn.Config(
    app,     # Port number
    reload=True      # Optional: enables auto-reload on code changes
)

server = uvicorn.Server(config=config)
await server.serve()

INFO:     Started server process [71353]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     10.110.3.178:57276 - "GET / HTTP/1.1" 200 OK
INFO:     192.168.13.158:58609 - "GET / HTTP/1.1" 200 OK
INFO:     192.168.13.158:58609 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     192.168.13.158:58609 - "GET / HTTP/1.1" 200 OK
INFO:     192.168.13.158:58609 - "GET /books HTTP/1.1" 200 OK
INFO:     192.168.13.158:58767 - "GET /books HTTP/1.1" 200 OK
INFO:     192.168.13.158:58784 - "GET /books HTTP/1.1" 200 OK
INFO:     192.168.13.158:58798 - "GET /books HTTP/1.1" 200 OK
